In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as MSE
import seaborn as sns
import xgboost
from pandas import read_csv
import pandas
import numpy as np
import matplotlib.pyplot as plt
import csv
from alys import save as SV
from sklearn.model_selection import train_test_split

In [ ]:
df_new = read_csv("new_Basetrain.csv", index_col =0)
df_kagg = read_csv("new_kaggtest.csv", index_col =0)

In [ ]:
ax = sns.lineplot(x="YearBuilt", y="target", data=df_new)

In [ ]:
df_ann = read_csv('Rapport/analyse2.csv') 
ax = sns.lineplot(x="YearBuilt", y="errorabs", data=df_ann)

In [ ]:
df_ann = read_csv('Rapport/analyse2.csv') 
ax = sns.lineplot(x="YearBuilt", y="pourcerror", data=df_ann)

In [ ]:
ax = sns.lineplot(x="YearBuilt", y="SalePriceLog", data=df_ann)

In [ ]:
corr_matrix = abs(df_new.corr())
corr_matrix['SalePriceLog'].sort_values(ascending=False).head(30)

In [ ]:
y_trainfull = df_new.SalePriceLog.values.reshape((1453,1))
x_trainfull = df_new.drop(['target','SalePriceLog'], axis=1)
x_test = df_kagg

In [ ]:
y_trainfull.shape,x_trainfull.shape,x_test.shape

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_trainfull,y_trainfull , test_size=0.3, random_state=42)

In [ ]:
x_train.shape, x_val.shape, y_train.shape, y_val.shape 

In [ ]:
xgb = xgboost.XGBRegressor(colsample_bytree=0.8, subsample=0.5,
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.8, n_estimators=2000,
                             reg_alpha=0.1, reg_lambda=0.3, gamma=0.01, 
                             silent=1, random_state =7, nthread = -1)


xgb.fit(x_train,y_train)
xgb_pred = xgb.predict(x_val)

In [ ]:
RMSE = np.sqrt(MSE(xgb_pred, y_val))
print(RMSE.round(4))

In [ ]:
y_test = xgb.predict(x_test)

In [ ]:
submission = pd.DataFrame({"Id": df_kagg.index,"SalePrice": np.exp(y_test)})
submission.to_csv("new_kagg6.csv", index= False)

In [ ]:
df_new.to_csv('new_Basetrain.csv')

In [ ]:
from IPython.display import Image
Image('png/idb8.png')

In [ ]:
df_new = read_csv("new_Basetrain.csv", index_col =0)
df_kagg = read_csv("new_kaggtest.csv", index_col =0)

In [ ]:
df_new['TotalSFlog'] = np.log(df_new.TotalSF) 
df_new['GrLivArealog'] = np.log(df_new.GrLivArea)

df_kagg['TotalSFlog'] = np.log(df_kagg.TotalSF)
df_kagg['GrLivArealog'] = np.log(df_kagg.GrLivArea)



In [ ]:
y_trainfull = df_new.SalePriceLog.values.reshape((1453,1))
x_trainfull = df_new.drop(['target','SalePriceLog'], axis=1)
x_test = df_kagg

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_trainfull,y_trainfull , test_size=0.3, random_state=42)

In [ ]:
#Operation trés longue, crash du noyeau si ont passe tout les paramétres.
from sklearn.model_selection import GridSearchCV

param_grid = [{'n_estimators': [400,500,600,700],'max_depth': [2,3,4,5],
               'colsample_bytree' : [0.5,0.6,0.7,0.8],'subsample' : [0.5,0.6,0.7,0.8],
#               'learning_rate':[0.04,0.05,0.06],'min_child_weight': [1.6,1.7,1.8],
#                'reg_alpha':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],'reg_lambda': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
#                'gamma': [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09],'silent': [1,2,3,4,5,6,7,8,9],
#                'random_state': [1,2,3,4,5,6,7,8,9], 'nthread' : [-2,-1.5,-1.25,-1,-0.75,-0.5,-0.25]
              }]
my_learn = xgboost.XGBRegressor()
grid_search = GridSearchCV(my_learn,param_grid, cv=5, scoring ='neg_mean_squared_error', return_train_score=True)
grid_search.fit(x_train,y_train)

In [ ]:
grid_search.best_params_

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_trainfull,y_trainfull , test_size=0.3, random_state=42)

In [ ]:
xgb = xgboost.XGBRegressor(colsample_bytree=0.5, subsample=0.7,
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.8, n_estimators=1000,
                             reg_alpha=0.1, reg_lambda=0.3, gamma=0.01, 
                             silent=1, random_state =7, nthread = -1)


xgb.fit(x_train,y_train)
xgb_pred = xgb.predict(x_val)

In [ ]:
RMSE = np.sqrt(MSE(xgb_pred, y_val))
print(RMSE.round(4))                            
#last 0.1145 avec max_depth 2 et n esti 1000
#last 0.1141 {'colsample_bytree': 0.5, 'max_depth': 3, 'n_estimators': 1000,'subsample': 0.7}

In [ ]:
y_test = xgb.predict(x_test)

In [ ]:
submission = pd.DataFrame({"Id": df_kagg.index,"SalePrice": np.exp(y_test)})
submission.to_csv("new_kagg7.csv", index= False)

In [ ]:
Le test suivant a fortement degradé le resulat kaggle, cependant je conserve le code en note